In [1]:
from models import ChatMessage, ChatHistory
import dspy
from lms.together import Together

from modules.chatter import ChatterModule

C:\Users\subho\anaconda3\envs\whisper2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lm = Together(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    temperature=0.5,
    max_tokens=1000,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1.2,
    stop=["<|eot_id|>", "<|eom_id|>", "\n\n---\n\n", "\n\n---", "---", "\n---"],
    # stop=["\n", "\n\n"],
)

In [3]:
dspy.settings.configure(lm=lm)

In [4]:
import json
import random
from models import ChatMessage, ChatHistory

# Load the dataset
with open('../training_data/conversations.json', 'r') as f:
    conversations = json.load(f)

# Set random seed for reproducibility
random.seed(42)
random.shuffle(conversations)

# Split into train (80%) and dev (20%) sets
split_idx = int(len(conversations) * 0.8)
train_data = conversations[:split_idx]
dev_data = conversations[split_idx:]

# Create trainset and devset with proper ChatHistory objects
def create_example(conv):
    # Create ChatHistory object with proper message format
    chat_history = ChatHistory(
        messages=[
            ChatMessage(**msg) for msg in conv['chat_history']['messages']
        ]
    )
    
    return dspy.Example({
        'chat_history': chat_history,
        'output': conv['output']
    }).with_inputs('chat_history')

trainset = [create_example(conv) for conv in train_data]
devset = [create_example(conv) for conv in dev_data]

print(f"Training examples: {len(trainset)}")
print(f"Dev examples: {len(devset)}")

Training examples: 8
Dev examples: 2


In [5]:
# Example from trainset
example = trainset[0]
print("Training Example Structure:")
print(example.chat_history)  # This is a ChatHistory object
print(example.chat_history.messages) 

Training Example Structure:
THE FAN: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
YOU: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
[ChatMessage(from_creator=False, content='I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?'), ChatMessage(from_creator=True, content='Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.'), ChatMessage(from_creator=False, content='It feels chaotic, but I also feel like Iâ€™m learning so much about myself.')]


In [14]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

# Create the KNN teleprompter
knn_teleprompter = KNNFewShot(7, trainset)

# Create an instance of ChatterModule
chatter = ChatterModule(examples=None)

# Compile with our ChatterModule
compiled_knn = knn_teleprompter.compile(chatter, trainset=trainset)

# Save the compiled model
#compiled_knn.save("saved_models/optimized_chatbot.json")

In [13]:
compiled_knn

responder.prog.predictor = Predict(StringSignature(chat_history -> reasoning, output
    instructions='You are an OnlyFans creator chatting on OnlyFans with a fan.\nYou are deciding on what your message should be.'
    chat_history = Field(annotation=ChatHistory required=True json_schema_extra={'desc': 'the chat history', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step to decide on our message. We", '__dspy_field_type': 'output', 'desc': '${reasoning}'})
    output = Field(annotation=str required=True json_schema_extra={'prefix': 'Your Message:', 'desc': 'the exact text of the message you will send to the fan.', '__dspy_field_type': 'output'})
))

In [9]:
compiled_knn

responder.prog.predictor = Predict(StringSignature(chat_history -> reasoning, output
    instructions='You are an OnlyFans creator chatting on OnlyFans with a fan.\nYou are deciding on what your message should be.'
    chat_history = Field(annotation=ChatHistory required=True json_schema_extra={'desc': 'the chat history', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step to decide on our message. We", '__dspy_field_type': 'output', 'desc': '${reasoning}'})
    output = Field(annotation=str required=True json_schema_extra={'prefix': 'Your Message:', 'desc': 'the exact text of the message you will send to the fan.', '__dspy_field_type': 'output'})
))

In [6]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

def metric(example, prediction, trace=None):
    return example.output == prediction.output

def test_knn(k_value):
    print(f"\nTesting with k={k_value}")
    print("-" * 50)
    
    bootstrap_args = {
        'metric': metric,
        'max_bootstrapped_demos': 2,
        'max_labeled_demos': 4,
        'max_rounds': 1,
        'max_errors': 5
    }

    knn_teleprompter = KNNFewShot(
        k=k_value,
        trainset=trainset,
        **bootstrap_args
    )

    chatter = ChatterModule(examples=None)
    compiled_knn = knn_teleprompter.compile(
        student=chatter,
        trainset=trainset
    )

    # Test with sample input
    test_history = trainset[0].chat_history
    result = compiled_knn(chat_history=test_history)
    
    # Get and print neighbors
    neighbors = knn_teleprompter.KNN(chat_history=test_history)
    
    print(f"\nInput conversation:")
    for msg in test_history.messages:
        role = "Fan" if not msg.from_creator else "Creator"
        print(f"{role}: {msg.content}")
    
    print(f"\nModel output: {result.output}")
    
    print(f"\nNearest {k_value} neighbors used:")
    for i, n in enumerate(neighbors):
        print(f"\nNeighbor {i+1}:")
        for msg in n.chat_history.messages:
            role = "Fan" if not msg.from_creator else "Creator"
            print(f"{role}: {msg.content}")
        print(f"Response: {n.output}")

# Test with different k values
for k in [2, 3, 4]:
    test_knn(k)


Testing with k=2
--------------------------------------------------


Please use standard predictors, e.g. dspy.Predict and dspy.ChainOfThought.
They now support type annotations and other features of TypedPredictors and tend to work much better out of the box.
Please let us know if you face any issues: https://github.com/stanfordnlp/dspy/issues
  0%|                                                                                        | 0/2 [00:00<?, ?it/s]WARNING:root:	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated, underperform, and are about to be deleted. ***
 		You are using the client Together, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.10s/it]

Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: You're navigating through life with courage, allowing yourself to learn and grow from each experience. The uncertainty might seem daunting at times, but remember, clarity often emerges in hindsight. Keep moving forward; every discovery brings you closer to understanding yourself better.

Nearest 2 neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel

100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [00:15<00:00,  5.23s/it]


Bootstrapped 0 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.

Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: The journey might seem messy right now, but every new insight brings clarity. Keep exploring; each rediscovery shapes who you'll become next.

Nearest 3 neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curiou

100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [00:24<00:00,  6.06s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: Try journaling these discoveries. Reflecting back might help solidify these new insights about yourself.

Nearest 4 neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 2:
Fan: Do you ever feel

In [7]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import random

class ResponseEvaluator:
    def __init__(self):
        self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        
    def calculate_metrics(self, prediction, reference):
        try:
            scores = self.scorer.score(reference, prediction)
            rouge_l = scores['rougeL'].fmeasure
        except:
            rouge_l = 0
            
        pred_emb = self.encoder.encode([prediction])[0]
        ref_emb = self.encoder.encode([reference])[0]
        semantic_sim = cosine_similarity([pred_emb], [ref_emb])[0][0]
        
        return {
            'rouge_l': rouge_l,
            'semantic_sim': semantic_sim
        }

def metric(example, prediction, trace=None):
    return example.output == prediction.output

def test_knn(k_value, trainset, num_samples=5):
    print(f"\nTesting with k={k_value}")
    print("-" * 50)
    
    bootstrap_args = {
        'metric': metric,
        'max_bootstrapped_demos': 2,
        'max_labeled_demos': 4,
        'max_rounds': 1,
        'max_errors': 5
    }

    knn_teleprompter = KNNFewShot(
        k=k_value,
        trainset=trainset,
        **bootstrap_args
    )

    chatter = ChatterModule(examples=None)
    compiled_knn = knn_teleprompter.compile(
        student=chatter,
        trainset=trainset
    )
    
    evaluator = ResponseEvaluator()
    metrics_list = []
    
    test_indices = random.sample(range(len(trainset)), num_samples)
    
    for idx in test_indices:
        test_example = trainset[idx]
        
        # Get model response
        result = compiled_knn(chat_history=test_example.chat_history)
        response = result.output
        
        # Calculate metrics
        metrics = evaluator.calculate_metrics(response, test_example.output)
        metrics_list.append(metrics)
        
        # Print detailed results for this example
        print(f"\nSample {idx + 1}:")
        print("Input (last message):", str(test_example.chat_history.messages[-1]))  # Using __str__ method
        print("Expected:", test_example.output)
        print("Generated:", response)
        print(f"ROUGE-L: {metrics['rouge_l']:.3f}")
        print(f"Semantic Similarity: {metrics['semantic_sim']:.3f}")
        
        # Print nearest neighbors
        neighbors = knn_teleprompter.KNN(chat_history=test_example.chat_history)
        print(f"\nNearest {k_value} neighbors used:")
        for i, n in enumerate(neighbors):
            print(f"\nNeighbor {i+1}:")
            print("Last message:", str(n.chat_history.messages[-1]))  # Using __str__ method
            print(f"Response: {n.output}")
    
    # Calculate average metrics
    avg_rouge = np.mean([m['rouge_l'] for m in metrics_list])
    avg_semantic = np.mean([m['semantic_sim'] for m in metrics_list])
    std_rouge = np.std([m['rouge_l'] for m in metrics_list])
    std_semantic = np.std([m['semantic_sim'] for m in metrics_list])
    
    print("\nOverall Metrics:")
    print(f"Average ROUGE-L: {avg_rouge:.3f} (±{std_rouge:.3f})")
    print(f"Average Semantic Similarity: {avg_semantic:.3f} (±{std_semantic:.3f})")
    
    return {
        'rouge_l': avg_rouge,
        'rouge_std': std_rouge,
        'semantic_sim': avg_semantic,
        'semantic_std': std_semantic
    }

# Test with different k values and collect results
results = {}
for k in [2, 3, 4]:
    results[k] = test_knn(k, trainset, num_samples=5)

# Print comparative summary
print("\nComparative Summary:")
print("-" * 50)
for k, metrics in results.items():
    print(f"\nk={k}:")
    print(f"ROUGE-L: {metrics['rouge_l']:.3f} (±{metrics['rouge_std']:.3f})")
    print(f"Semantic Similarity: {metrics['semantic_sim']:.3f} (±{metrics['semantic_std']:.3f})")


Testing with k=2
--------------------------------------------------


100%|█████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.61s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Sample 1:
Input (last message): THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Expected: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.
Generated: The journey might seem messy now, but every revelation brings you closer to understanding yourself better. Keep exploring!
ROUGE-L: 0.114
Semantic Similarity: 0.367

Nearest 2 neighbors used:

Neighbor 1:
Last message: THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 2:
Last message: THE FAN: I like that. Like, I can just be me but maybe a little less chaotic?
Response: Exactly! Improvement isnâ€™t about fixing, just embracing yourself with more awareness and acceptance.


100%|█████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.19s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Sample 5:
Input (last message): THE FAN: I love that idea. Maybe right now, Iâ€™m in my â€˜mystical scienceâ€™ chapter.
Expected: Exactly! Allow yourself to dive deeply into each chapter. Your interests are like seasonsâ€”they come and go with purpose.
Generated: That sounds fascinating! Dive deep into mystical science for now; remember, future chapters await when the time comes. Learning unfolds over years, so enjoy this journey without worrying about missing anything else.
ROUGE-L: 0.154
Semantic Similarity: 0.422

Nearest 2 neighbors used:

Neighbor 1:
Last message: THE FAN: I love that idea. Maybe right now, Iâ€™m in my â€˜mystical scienceâ€™ chapter.
Response: Exactly! Allow yourself to dive deeply into each chapter. Your interests are like seasonsâ€”they come and go with purpose.

Neighbor 2:
Last message: THE FAN: Iâ€™ll try focusing on my small joysâ€”like vintage records and herbal teas.


...

Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Sample 3:
Input (last message): THE FAN: Thatâ€™s really comforting. I always thought connection had to be extroverted.
Expected: Not at all. Itâ€™s about quality, not quantity. Authentic connection doesnâ€™t demand you to be someone youâ€™re not.
Generated: Shared passions can create strong bonds without needing loud environments. Your unique interests are bridges to others; let them connect you organically.
ROUGE-L: 0.093
Semantic Similarity: 0.105

Nearest 4 neighbors used:

Neighbor 1:
Last message: THE FAN: Thatâ€™s really comforting. I always thought connection had to be extroverted.
Response: Not at all. Itâ€™s about quality, not quantity. Authentic connection doesnâ€™t demand you to be someone youâ€™re not.

Neighbor 2:
Last message: THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it m

100%|█████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.15s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Sample 1:
Input (last message): THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Expected: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.
Generated: Keep exploring! Every new discovery brings clarity. And don't worry if things seem messy along the way; that's where some of the best insights come from.
ROUGE-L: 0.178
Semantic Similarity: 0.395

Nearest 4 neighbors used:

Neighbor 1:
Last message: THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 2:
Last message: THE FAN: I like that. Like, I can just be me but maybe a little less chaotic?
Response: Exactly! Improvement isnâ€™t about fixing, just embracing yourself with more awareness and acceptance.

Neighbor 3:
Last

100%|█████████████████████████████████████████████████████| 4/4 [00:21<00:00,  5.25s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Sample 4:
Input (last message): THE FAN: Thatâ€™s really reassuring. Sometimes I just want to escape into another time.
Expected: Thatâ€™s understandable. Just remember that you can bring elements of those eras into your life now.
Generated: Escapism isn't inherently bad; it can actually inspire new perspectives. The key is balance between escaping and engaging with reality.
ROUGE-L: 0.053
Semantic Similarity: 0.206

Nearest 4 neighbors used:

Neighbor 1:
Last message: THE FAN: Thatâ€™s really reassuring. Sometimes I just want to escape into another time.
Response: Thatâ€™s understandable. Just remember that you can bring elements of those eras into your life now.

Neighbor 2:
Last message: THE FAN: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 3:
Last m

In [8]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

def metric(example, prediction, trace=None):
    return example.output == prediction.output

# Create the KNN teleprompter with optimal parameters
bootstrap_args = {
    'metric': metric,
    'max_bootstrapped_demos': 2,
    'max_labeled_demos': 4,
    'max_rounds': 1,
    'max_errors': 5
}

# Initialize KNN with k=2
knn_teleprompter = KNNFewShot(
    k=4,  # Using 2 nearest neighbors
    trainset=trainset,
    **bootstrap_args
)

# Create and compile
chatter = ChatterModule(examples=None)
compiled_knn = knn_teleprompter.compile(
    student=chatter,
    trainset=trainset
)

# Optional: Save the compiled model
compiled_knn.save("saved_models/optimized_chatbot.json")

# Test the model (optional)
def test_model():
    test_history = trainset[0].chat_history
    result = compiled_knn(chat_history=test_history)
    
    print("\nTest Results:")
    print("Input conversation:")
    for msg in test_history.messages:
        role = "Fan" if not msg.from_creator else "Creator"
        print(f"{role}: {msg.content}")
    
    print(f"\nModel output: {result.output}")
    
    neighbors = knn_teleprompter.KNN(chat_history=test_history)
    print(f"\nNearest neighbors used:")
    for i, n in enumerate(neighbors):
        print(f"\nNeighbor {i+1}:")
        for msg in n.chat_history.messages:
            role = "Fan" if not msg.from_creator else "Creator"
            print(f"{role}: {msg.content}")
        print(f"Response: {n.output}")

# Uncomment to test
test_model()

100%|█████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.09s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Test Results:
Input conversation:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.

Model output: Keep exploring without judgment. Every version of you has value; don't compare, integrate.

Nearest neighbors used:

Neighbor 1:
Fan: I feel like Iâ€™m always â€˜rediscoveringâ€™ myself. Is that normal, or am I just indecisive?
Creator: Rediscovery is a sign of growth. Weâ€™re constantly evolving, and itâ€™s healthy to question who you are.
Fan: It feels chaotic, but I also feel like Iâ€™m learning so much about myself.
Response: Thatâ€™s the beauty of self-discovery. Embrace the chaosâ€”it means youâ€™re alive and curious.

Neighbor 2:
Fan: Do you ever feel l

In [8]:
compiled_knn

responder.prog.predictor = Predict(StringSignature(chat_history -> reasoning, output
    instructions='You are an OnlyFans creator chatting on OnlyFans with a fan.\nYou are deciding on what your message should be.'
    chat_history = Field(annotation=ChatHistory required=True json_schema_extra={'desc': 'the chat history', '__dspy_field_type': 'input', 'prefix': 'Chat History:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step to decide on our message. We", '__dspy_field_type': 'output', 'desc': '${reasoning}'})
    output = Field(annotation=str required=True json_schema_extra={'prefix': 'Your Message:', 'desc': 'the exact text of the message you will send to the fan.', '__dspy_field_type': 'output'})
))

In [9]:
chatter = ChatterModule(examples=None)  # Create a new instance
chatter.load(path="saved_models/optimized_chatbot.json")  # Load the optimized version

chat_history = ChatHistory()

while True:
    # Get user input
    user_input = input("You: ")

    # Append user input to chat history
    chat_history.messages.append(
        ChatMessage(
            from_creator=False,
            content=user_input,
        ),
    )

    # Send request to endpoint using loaded model
    response = chatter(chat_history=chat_history).output

    # Append response to chat history
    chat_history.messages.append(
        ChatMessage(
            from_creator=True,
            content=response,
        ),
    )
    # Print response
    print()
    print("Response:", response)
    print()

You:  who are you?



Response: Hi there! I'm [Your Name], nice to meet you! Welcome to my page! What drew you here? Are you looking for something specific or just browsing?



You:  What is your name?



Response: My name is [Your Actual Name]. It's lovely to have you here! Is there anything else you'd like to know about me?



KeyboardInterrupt: Interrupted by user